This tutorial describes the basics of assigning users to projects, workspaces, and specific tasks in Label Studio Enterprise using the Python SDK.

**Note:** This code utilizes functions from an older version of the Label Studio SDK (v0.0.34).
The newer versions v1.0 and above still support the functionalities of the old version, but you will need to specify
[`label_studio_sdk._legacy`](../../README.md) in your script.

## Connect to Label Studio Enterprise

Start by connecting to Label Studio Enterprise using the `Client` module:

In [ ]:
import label_studio_sdk

ls = label_studio_sdk.Client('http://localhost:8080', api_key='')

Get all users

In [ ]:
users = ls.get_users()

Get project by ID

In [ ]:
PROJECT_ID = 248 # change with your project id
project = ls.get_project(id=PROJECT_ID)

Get project members

In [ ]:
members = project.get_members()

## Add user as a project member

To find out who is already part of your organization, you can list all users that are members of your current organization:

In [ ]:
user = users[0]
res = project.add_member(user)

## Change user role

You can change the role of a user in the currently active organization.

In [ ]:
from label_studio_sdk._legacy.users import UserRole

user.set_role(UserRole.REVIEWER)
print(user)

## Assign tasks to annotators

You can assign tasks to annotators so that they can work on specific tasks. You must be using manual task distribution in your project to assign tasks.

In [ ]:
tasks_ids = project.get_tasks(only_ids=True)
project.assign_annotators(users, tasks_ids)

## Assign tasks to reviewers

You can assign tasks to reviewers so that they can review the annotations for specific tasks.

In [ ]:
project.assign_reviewers(users, tasks_ids)

## List all tasks assigned to specific annotators

Before you assign new tasks to annotators, you might want to know which or how many tasks an annotator already has to work on. To list all the tasks already assigned to an annotator, run the following code:

In [ ]:
from label_studio_sdk.data_manager import Filters, Column, Operator, Type

filters = Filters.create(Filters.OR, [
    Filters.item(
        Column.annotators,
        Operator.CONTAINS,
        Type.List,
        Filters.value(user.id)
    )
])
tasks = project.get_tasks(filters=filters)
print(tasks)

## List all tasks assigned to specific reviewers

Before you assign new tasks to reviewers, you might want to see which tasks or how many tasks a reviewer is assigned to already. To list all tasks already assigned to a reviewer, run the following code:

In [ ]:
from label_studio_sdk.data_manager import Filters, Column, Operator, Type

filters = Filters.create(Filters.OR, [
    Filters.item(
        Column.reviewers,
        Operator.CONTAINS,
        Type.List,
        Filters.value(user.id)
    )
])
tasks = project.get_tasks(filters=filters)
print(tasks)

## Remove annotators from tasks

To remove annotators from a task, for example to change who is assigned to a specific task, or remove an annotator that no longer works on the project, run the following code:

In [ ]:
tasks_ids = project.get_tasks(only_ids=True)
project.delete_annotators_assignment(tasks_ids)

## Remove reviewers from tasks

To remove reviewers from a task, run the following code:

In [ ]:
tasks_ids = project.get_tasks(only_ids=True)
project.delete_reviewers_assignment(tasks_ids)

## Randomly assign users to the dataset

In [ ]:
project.assign_annotators_by_sampling(members)